In [0]:
%pip install mlflow==2.10.1 lxml==4.9.3 langchain==0.1.5 databricks-vectorsearch==0.22 cloudpickle==2.2.1 databricks-sdk==0.18.0 cloudpickle==2.2.1 pydantic==2.5.2 openpyxl
%pip install pip mlflow[databricks]==2.10.1 PyPDF2

dbutils.library.restartPython()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window
import pandas as pd
import random
import os
import uuid
import json
import io
import plotly.express as px
from langchain_community.chat_models import ChatDatabricks
import PyPDF2

from datetime import datetime
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType

In [0]:
chat_endpoint = "databricks-llama-4-maverick" 
chat_model = ChatDatabricks(endpoint = chat_endpoint,
                            temperature = 0,
                            max_tokens = 8192) 

In [0]:
base_path = "/Workspace/Users/elielriey@gmail.com/GenAILearning/Databricks_GenAIEngineering/PromptEngineering/papers"
files_list = os.listdir(base_path)

In [0]:
def generate_extraction_prompt(topic, article):

    EXTRATION_PROMPT = f"""
    INSTRUCTION:
    You will receive a text with some topics and subtopics. You task is to extract only the topic you will be provided in the task:

    TASK:
    Extract the whole text of the {topic} of the article.

    OUTPUT:
    Your answer must be only the extracted text. 
    Do not add any other text or information.
    Do not explain what you do.

    Following the instrucionts above extract the {topic} of this article: {article}
    """

    return EXTRATION_PROMPT

def generate_summarization_prompt(topic): 

    SUMMARIZE_PROMPT= """
    INSTRUCTION:  
    You are a research assistant specialized in summarizing and analyzing scientific articles. you will be provides of a {topic} of a article. You will analise it in details.

    CONTEXT:  
    The content of the {topic} are in the context.

    INPUT / QUESTION:  
    context: {topic}

    TASK: 
    Summarize the text present in the context, bringing the main points presented and the references used to support the points. 
    """

    return SUMMARIZE_PROMPT

In [0]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text  # Truncate if needed for token limit

# def analyze_article(EXTRACTION_PROMPT, SUMMARIZE_PROMPT):
#     extraction = chat_model.invoke(EXTRACTION_PROMPT).content
#     SUMMARIZE_PROMPT = generate_summarization_prompt(extraction)
#     sumarization = chat_model.invoke(SUMMARIZE_PROMPT)
            
#     return sumarization.content

# def main():
#     for filename in files_list:
        
#         if filename.endswith(".pdf"):
#             path = os.path.join(base_path, filename)
#             article_text = extract_text_from_pdf(path)
#             for topic in ["Introduction"]:#, "Methods", "Results", "Discussion"]:
#                 analysis = analyze_article(generate_extraction_prompt(topic, article_text))
#                 output_path = path.replace(".pdf", "_analysis.txt")
#                 with open(output_path, "w", encoding="utf-8") as out_file:
#                     out_file.write(analysis)
#                 print(f"Analysis saved to {output_path}")

In [0]:
topic = "Introduction"
print(f"Analisando o artigo {files_list[0]}\n")
path = os.path.join(base_path, files_list[0])
article = extract_text_from_pdf(path)
extraction = chat_model.invoke(generate_extraction_prompt(topic, article)).content
extraction


In [0]:
main()